# ABIA Proyect 1: Hill Climbing

<hr>
<hr>

## Imports

In [66]:
import time
import timeit
import random
import numpy as np
from tkinter import Y
from math import trunc, log
from typing import List, Generator, Set
from search import Problem, hill_climbing

<hr>
<hr>

## File: abia_energia.py

### Constants

In [3]:
# Tipos de cliente
CLIENTE_XG = 0
CLIENTE_MG = 1
CLIENTE_G = 2

# Tipos de central
CENTRAL_A = 0
CENTRAL_B = 1
CENTRAL_C = 2

# Prioridades
GARANTIZADO = 0
NOGARANTIZADO = 1

# Producción por tipo de central
PROD = [[500.0, 250.0], [150.0, 100.0], [90.0, 10.0]]

# Consumo por tipo de cliente
CONSUMOS = [[15.0, 5.0], [3.0, 2.0], [2, 1]]

# Tabla de precios por tipo de cliente
# Primera dimensión: tipo de cliente
# Segunda dimensión, por orden: garantizado, no garantizado, indemnización
PRECIOS = [[40.0, 30.0, 5.0], [50.0, 40.0, 5.0], [60.0, 50.0, 5.0]]

# Tabla de costes
# Primera dimensión: tipo de central
# Segunda dimensión, por orden: coste por producción, coste diario, coste parada
COSTES = [[5, 2000, 1500], [8, 1000, 500], [15, 500, 150]]

# Tabla de pérdidas
# En cada posición, el primer valor es la distancia máxima del rango, el segundo valor es el tanto por uno de pérdida.
PERDIDA = [[10, 0], [25, 0.1], [50, 0.2], [75, 0.4], [1000, 0.6]]
TIPO = [0, 1, 2]
TIPOCL = [0, 1, 2]
TIPOCNT = [0, 1]

<hr>

### Class Central

In [4]:
class Central(object):

    def __init__(self, tipo: int, produccion: float, cx: int, cy: int):
        self.Tipo = tipo
        self.Produccion = produccion
        self.CoordX = cx
        self.CoordY = cy

    def __repr__(self) -> str:
        return f"[Tipo={self.Tipo}|Produccion={self.Produccion}|cx={self.CoordX}|cy={self.CoordY}]"

### Class Centrals

In [5]:
class Centrales(list):
    
    def __init__(self, centrales_por_tipo: List[int], seed: int):
        if len(centrales_por_tipo) != 3:
            raise Exception("Vector Centrales de tamaño incorrecto")
        v_centrales = []
        rand = random.Random(seed)
        for i in range(3):
            for j in range(centrales_por_tipo[i]):
                p = (rand.random() * PROD[i][0]) + PROD[i][1]
                c = Central(TIPO[i], trunc(p), rand.randint(
                    0, 100), rand.randint(0, 100))
                v_centrales.append(c)
        super().__init__(v_centrales)

<hr>

### Class Client

In [6]:
class Cliente(object):
    
    def __init__(self, t: int, cons: float, cont: int, cx: int, cy: int):
        self.Tipo = t
        self.Consumo = cons
        self.Contrato = cont
        self.CoordX = cx
        self.CoordY = cy

    def __repr__(self) -> str:
        return f"[Tipo={self.Tipo}|Consumo={self.Consumo}|Contrato={self.Contrato}|CoordX={self.CoordX}|CoordY={self.CoordY}]"

### Class Clients

In [7]:
class Clientes(list):

    def __init__(self, ncl: int, propc: List[float], propg: float, seed: int):
        if len(propc) != 3:
            raise Exception(
                "Vector proporciones tipos clientes de tamaño incorrecto")
        if (propc[0] + propc[1] + propc[2]) != 1.0:
            raise Exception("Vector proporciones tipos clientes no suma 1")
        if 0.0 > propg > 1.0:
            raise Exception("Proporcion garantizado fuera de limites")

        v_clientes = []
        rand = random.Random(seed + 1)

        for i in range(ncl):
            dice = rand.random()
            if dice < propc[0]:
                rd = 0
            elif dice < (propc[0] + propc[1]):
                rd = 1
            else:
                rd = 2

            dice = rand.random()
            if dice < propg:
                rc = 0
            else:
                rc = 1

            c = (rand.random() * CONSUMOS[rd][0]) + CONSUMOS[rd][1]
            v_clientes.append(
                Cliente(TIPOCL[rd], trunc(c), TIPOCNT[rc], rand.randint(0, 100), rand.randint(0, 100)))
        super().__init__(v_clientes)

<hr>

### Static Class VEnergia

In [8]:
class VEnergia(object):

    @staticmethod
    def tarifa_cliente_garantizada(tipo: int) -> float:
        
        """ Retorna la tarifa de un cliente con servicio garantizado. """

        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return PRECIOS[tipo][0]

    @staticmethod
    def tarifa_cliente_no_garantizada(tipo: int) -> float:
        
        """ Retorna la tarifa de un cliente con servicio no garantizado. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return PRECIOS[tipo][1]

    @staticmethod
    def tarifa_cliente_penalizacion(tipo: int) -> float:
        
        """ Retorna la penalización por servir a un cliente con servicio no garantizado. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return PRECIOS[tipo][2]

    @staticmethod
    def prices_production_mw(tipo: int) -> float:
        
        """ Retorna el coste de producción por MW para una central de un tipo. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return COSTES[tipo][0]

    @staticmethod
    def daily_cost(tipo: int) -> float:
        
        """ Retorna el coste de una central en marcha segun su tipo. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return COSTES[tipo][1]

    @staticmethod
    def stop_cost(tipo: int) -> float:
        
        """ Retorna el coste de una central en parada segun su tipo. """
        
        if tipo < 0 or tipo > 2:
            raise Exception("Tipo fuera de rango")
        else:
            return COSTES[tipo][2]

    @staticmethod
    def loss(distancia: float) -> float:
        
        """ Retorna la perdida (en tanto por uno) segun la distancia entre central y cliente. """
        
        i = 0
        while distancia > PERDIDA[i][0]:
            i = i + 1
        return PERDIDA[i][1]

<hr>
<hr>

## File: abia_energy_problem.py

### Class Problem Parameters

Esta clase es la misma desde que se crea, por tanto, ¿podríamos evitarnos pasarla por cada copia? 

In [9]:
class ProblemParameters(object):
    def __init__(self, clients_vector: Clientes, power_plants_vector: Centrales) -> None:
        self.clients_vector = clients_vector
        self.power_plants_vector = power_plants_vector

    def __repr__(self):
        return f"clients_vector={self.clients_vector}\n\npower_plants_vector={self.power_plants_vector})"

<hr>

### Operators

In [10]:
class Operator(object):
    pass

class MoveClient(Operator):
    def __init__(self, id_client, id_destination_PwP):
        self.id_client = id_client
        self.id_destination_PwP = id_destination_PwP

    def __repr__(self) -> str:
        return f"Client {self.id_client} has been moved to power plant {self.id_destination_PwP}"

class SwapClients(Operator):
    def __init__(self, id_client1, id_client2):
        self.id_client1 = id_client1
        self.id_client2 = id_client2

    def __repr__(self) -> str:
        return f"Swap between Client {self.id_client1} and Client {self.id_client2}"

class RemoveNGClient(Operator):
    def __init__(self, id_client: int):
        self.id_client = id_client

    def __repr__(self) -> str:
        return f"Client {self.id_client} has been removed"

<hr>

### Our methods

Se podria meter en VEnergia?

In [11]:
def distance(obj1, obj2):
    x = obj1.CoordX - obj2.CoordX
    y = obj1.CoordY - obj2.CoordY
    return (x**2 + y ** 2)**(1/2)

def electry_supplied_to_client(client: Cliente, power_plant: Central) -> int:
    dist = distance(client, power_plant)
    return client.Consumo * (1 + VEnergia.loss(dist))

<hr>

### Class State Representation

In [12]:
class StateRepresentation(object):
    
    def __init__(self, 
                 params: ProblemParameters, 
                 c_pp: List[int], 
                 remain: List[float], 
                 consum: List[List[float]], 
                 gain: float,
                 entropy:float,
                 prices: List[List[float]]) -> None:

        self.params = params
        self.c_pp   = c_pp
        self.remain = remain
        self.consum = consum
        self.prices = prices
        self.gain   = gain
        self.entropy = entropy

    def copy(self):
        return StateRepresentation(self.params, 
                                   self.c_pp.copy(), 
                                   self.remain.copy(), 
                                   self.consum, 
                                   self.gain,
                                   self.entropy,
                                   self.prices)

    def __repr__(self) -> str:
        s_c = 0
        for i in self.c_pp:
            if i != -1: s_c += 1
        return f"Serviced clietns = {s_c} of {len(self.c_pp)} \nGains: {self.heuristic()} \n"
        
    def generate_actions(self) -> Generator[Operator, None, None]:
        num_c  = len(self.params.clients_vector)
        num_pp = len(self.params.power_plants_vector)

        for id_c1 in range(num_c):
            for id_pp in range(num_pp):
                if id_pp == self.c_pp[id_c1]:
                    continue

                c_consum = self.consum[id_pp][id_c1]
                if c_consum < self.remain[id_pp]:
                    yield MoveClient(id_c1, id_pp)

            if id_c1 != num_c - 1:
                for id_c2 in range(id_c1 + 1, num_c):
                    id_pp1 = self.c_pp[id_c1]
                    id_pp2 = self.c_pp[id_c2]

                    if id_pp1 == id_pp2 or id_pp1 == -1 or id_pp2 == -1:
                        continue

                    consum_c1_pp1 = self.consum[id_pp1][id_c1]
                    consum_c1_pp2 = self.consum[id_pp2][id_c1]
                    consum_c2_pp1 = self.consum[id_pp1][id_c2]
                    consum_c2_pp2 = self.consum[id_pp2][id_c2]

                    remain_pp1 = self.remain[id_pp1]
                    remain_pp2 = self.remain[id_pp2]

                    if consum_c2_pp1 - consum_c1_pp1 < remain_pp1 and consum_c1_pp2 - consum_c2_pp2 < remain_pp2:
                        yield SwapClients(id_c1, id_c2)

    def apply_action(self, action: Operator):
        new_state = self.copy()

        if isinstance(action, MoveClient):
            id_c   = action.id_client
            id_pp1 = self.c_pp[id_c] 
            pp1    = self.params.power_plants_vector[id_pp1]
            id_pp2 = action.id_destination_PwP
            pp2    = self.params.power_plants_vector[id_pp2]

            new_state.c_pp[id_c] = id_pp2

            if id_pp1 != -1:
                new_state.remain[id_pp1] += self.consum[id_pp1][id_c]
                new_state.entropy 
                
                if new_state.remain[id_pp1] == pp1.Produccion:
                    new_state.gain += self.prices[1][id_pp1] - self.prices[0][id_pp1]
    
            else:
                new_state.gain += self.prices[2][id_c]

            if new_state.remain[id_pp2] == pp2.Produccion:
                new_state.gain -= self.prices[1][id_pp2] + self.prices[0][id_pp2]

            new_state.remain[id_pp2] -= self.consum[id_pp2][id_c]

        elif isinstance(action, SwapClients):
            id_c1  = action.id_client1
            id_c2  = action.id_client2
            id_pp1 = self.c_pp[id_c1]
            id_pp2 = self.c_pp[id_c2]

            new_state.c_pp[id_c1] = id_pp2
            new_state.c_pp[id_c2] = id_pp1

            new_state.remain[id_pp1] += self.consum[id_pp1][id_c1] - self.consum[id_pp1][id_c2]
            new_state.remain[id_pp2] += self.consum[id_pp2][id_c2] - self.consum[id_pp2][id_c1]

        return new_state

    def heuristic(self) -> float:
        return self.gain

<hr>

### Generation of the initial state

In [67]:
class InitialState(object):

    @staticmethod
    def simple_state(params) -> StateRepresentation:
        remain, consum = InitialState.remain_consum(params)
        
        c_pp  = []
        pp    = list(range(len(params.power_plants_vector)))
        id_pp = pp.pop()
        for id_client, client in enumerate(params.clients_vector):
            if client.Contrato == NOGARANTIZADO:
                c_pp.append(-1)
                continue

            c_consum = consum[id_pp][id_client]
            while True:
                if c_consum < remain[id_pp]:
                    c_pp.append(id_pp)
                    remain[id_pp] -= c_consum
                    break
                id_pp = pp.pop()

        c_pp = np.array(c_pp)
        
        gain, prices = InitialState.calculate_gain(params, c_pp, remain)
        return StateRepresentation(params, c_pp, remain, consum, gain, prices)
    
    @staticmethod
    def complex_state(params, worst=False) -> StateRepresentation:
        remain, consum = InitialState.remain_consum(params)
        
        c_pp   = []
        num_c  = len(params.clients_vector)
        num_pp = len(params.power_plants_vector)
        id_c   = 0
        id_pp  = 0
        cycle  = 0

        MAX_NUM_CYCLE = 15
        while id_c < num_c:
            if params.clients_vector[id_c].Tipo == NOGARANTIZADO:
                c_pp.append(-1)

            elif remain[id_pp] > consum[id_pp][id_c]:
                remain[id_pp] -= consum[id_pp][id_c]
                c_pp.append(id_pp)
            else:
                id_pp += 1

                if id_pp == num_pp:
                    id_pp = 0
                    cycle += 1
                    if cycle == MAX_NUM_CYCLE:
                        raise Exception("Sorry, max number of cycles did it")
                continue

            if worst:
                id_pp += 1
                if id_pp == num_pp:
                    id_pp = 0
                    cycle += 1
                    if cycle == MAX_NUM_CYCLE:
                        raise Exception("Sorry, max number of cycles did it")
            id_c += 1

        c_pp = np.array(c_pp)

        gain, prices = InitialState.calculate_gain(params, c_pp, remain)

        return StateRepresentation(params, c_pp, remain, consum, gain, prices)

    @staticmethod
    def remain_consum(params):
        remain = []
        consum = []

        for pp in params.power_plants_vector:
            remain.append(pp.Produccion)
            row = []
            for client in params.clients_vector:
                c_consum = electry_supplied_to_client(client, pp)
                row.append(c_consum)
            consum.append(row)

        remain = np.array(remain)
        consum = np.array(consum)
        return remain, consum

    @staticmethod
    def calculate_gain(params, c_pp, remain) -> float:
        gain = 0
        prices = [[], [], []]

        for id_pp, pp in enumerate(params.power_plants_vector):
            prices[0].append(VEnergia.stop_cost(pp.Tipo))
            prices[1].append(VEnergia.daily_cost(pp.Tipo) + VEnergia.prices_production_mw(pp.Tipo) * pp.Produccion)

            if pp.Produccion == remain[id_pp]: 
                gain -= prices[0][id_pp]
            else:
                gain -= prices[1][id_pp]

        for id_c, c in enumerate(params.clients_vector):
            if c.Contrato == 0:   
                gain += c.Consumo * VEnergia.tarifa_cliente_garantizada(c.Tipo)
            else:                 
                gain += c.Consumo * VEnergia.tarifa_cliente_no_garantizada(c.Tipo)

            prices[2].append(c.Consumo * VEnergia.tarifa_cliente_penalizacion(c.Tipo))
            if c_pp[id_c] == -1:  gain -= prices[2][id_c]

        return gain, prices

    @staticmethod
    def calculate_entropy(params: ProblemParameters, remaining_energies):
        total_entropy = 0
        for id, remain in enumerate(remaining_energies):
            max_prod = params.power_plants_vector[id].Produccion
            occupancy = 1 - (remain/max_prod)
            if occupancy > 0:
                total_entropy = total_entropy - (occupancy * log(occupancy))
        return -total_entropy
        

<hr>

### Class of the Problem

In [63]:
class EnergyProblem(Problem):
    def __init__(self, initial_state: StateRepresentation):
        self.times = [0, 0, 0]
        self.iterations = [0, 0, 0]
        super().__init__(initial_state)

    def actions(self, state: StateRepresentation) -> Generator[Operator, None, None]:
        print(calculate_entropy(state.params, state.remain))
        print(state.heuristic())
        print(state.remain)
        print(sum(state.remain))
        start   = time.time()
        actions = state.generate_actions()
        end     = time.time()
        self.times[0] += end - start
        self.iterations[0] += 1
        return actions

    def result(self, state: StateRepresentation, action: Operator) -> StateRepresentation:
        start   = time.time()
        result  = state.apply_action(action)
        end     = time.time()
        self.times[1] += end - start
        self.iterations[1] += 1
        return result

    def value(self, state: StateRepresentation) -> float:
        start   = time.time()
        value   = state.gain
        end     = time.time()
        self.times[2] += end - start
        self.iterations[2] += 1
        return value

    def goal_test(self, state: StateRepresentation) -> bool:
        return False

<hr>
<hr>

## Execution of the program

In [64]:
clientes       = Clientes(ncl=1000, propc=[0.25, 0.3, 0.45], propg=0.75, seed=1234)
centrales      = Centrales(centrales_por_tipo=[4, 20, 25], seed=1234)
parametros     = ProblemParameters(clients_vector=clientes, power_plants_vector=centrales)
estado_inicial = InitialState.simple_state(parametros)
problem        = EnergyProblem(estado_inicial)


In [65]:
print(estado_inicial)

start     = time.time()
ejecucion = hill_climbing(problem)
end       = time.time()

print(ejecucion)

print(f"Time taken: {end - start}s")

Serviced clietns = 754 of 1000 
Gains: 104927.0 

-1.9620374675120003
104927.0
[733 253 585  27   2   1   6   9  10   1   3   2  14   4   0   1  11  13
   1   0   1   1   8   4   0  25   3  14   4   1   1  10   1   4  10   0
   0   2   1   0   2   1  16  14  24   1   2   1   1]
1828
-1.6928397905104033
105022.0
[733 253 585  27   2   1   6   9  10   1   3   2  14   4   0   1  11  13
   1   0   1   1   8   4   0  25   3  14   4   1   1  10   1   4  10   0
   0   2   1   0   2   1  16  14   1   1   2   1   1]
1805
-1.657407769661657
105117.0
[733 253 585   4   2   1   6   9  10   1   3   2  14   4   0   1  11  13
   1   0   1   1   8   4   0  25   3  14   4   1   1  10   1   4  10   0
   0   2   1   0   2   1  16  14   1   1   2   1   1]
1782
-1.5375936035174618
105177.0
[733 253 585   4   2   1   6   9  10   1   3   2   0   4   0   1  11  13
   1   0   1   1   8   4   0  25   3  14   4   1   1  10   1   4  10   0
   0   2   1   0   2   1  16  14   1   1   2   1   1]
1768
-1.387264568918

<hr>

### Analizing problems

Hemos añadido codigo en la clase EnergyProblem que cronometra el tiempo de cada ejecución de las funciones que tenemos y cuenta cuantas veces se ejecuta.

In [ ]:
print(f"Generate actions: {problem.times[0]}s, {problem.iterations[0]} times")
print(f"Apply action: {problem.times[1]}s, {problem.iterations[1]} times")
print(f"Heuristic: {problem.times[2]}s, {problem.iterations[2]} times")

Generate actions: 0.0s, 63 times
Apply action: 70.42881631851196s, 6580307 times
Heuristic: 1.9241681098937988s, 6580433 times


Tenemos que reducir el tiempo de ejecución de <i style="color: #ffa530">apply_actions()</i> ya que dura demasiado.

In [ ]:
print(timeit.timeit(lambda: estado_inicial.generate_actions(), number=1))
print(timeit.timeit(lambda: estado_inicial.apply_action(MoveClient(0, 1)), number=1))
print(timeit.timeit(lambda: estado_inicial.apply_action(SwapClients(0, 1)), number=1))
print(timeit.timeit(lambda: estado_inicial.heuristic(), number=1))

4.499976057559252e-06
5.879998207092285e-05
3.1400006264448166e-05
2.2999593056738377e-06
